In [ ]:
#!pip install langchain

In [ ]:
#!pip install openai

In [ ]:
#!pip install huggingface-hub

In [ ]:
#!pip install streamlit

In [ ]:
#!pip install streamlit-chat

In [ ]:
#!pip install langchain_community

In [ ]:
#!pip install langchain-openai

In [ ]:
#!pip install pypdf

In [ ]:
#!pip install tiktoken

In [ ]:
#!pip install sentence-transformers

In [ ]:
#!pip install unstructured

In [ ]:
#!pip install chromadb

In [15]:
import streamlit as st
from streamlit_chat import message
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
from langchain import PromptTemplate
import os

# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = "sk-"

# 로컬 PDF 파일 경로
pdf_path = "c:/data/DNN.pdf"  # 로컬 경로를 적절히 변경하세요

# PDF 로드 및 청크 분할
def load_and_split_pdf(pdf_path, chunk_size=1000, chunk_overlap=20):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

# 문서 청크 로드
docs = load_and_split_pdf(pdf_path)

# 문서 벡터 저장소 설정
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(docs, embeddings, persist_directory="c:/data")

# ConversationalRetrievalChain 설정
chain = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(temperature=0.0, model_name='gpt-4'),
    retriever=db.as_retriever()
)


###pdf 요약에 관련된 부분
prompt1 = PromptTemplate(
    input_variables=['docs'],
    template="다음 문장을 3문장으로 요약하세요.\n\n{docs}"
)

llm = ChatOpenAI(temperature=0,  # 창의성 0으로 설정 
                 max_tokens=2048,  # 최대 토큰수
                 model_name='gpt-4',  # 모델명
                )

chain1 = LLMChain(llm=llm, prompt=prompt1, output_key="pdf_summary")
result=chain1.run(docs=docs)

# Streamlit 앱 설정
st.title("심층신경망개론 챗봇")
st.subheader("강의 요약")
st.write(result)

# 대화 기록 초기화
if 'history' not in st.session_state:
    st.session_state['history'] = []

if 'generated' not in st.session_state:
    st.session_state['generated'] = ["안녕하세요! 강의에 관해 질문해보세요."]

if 'past' not in st.session_state:
    st.session_state['past'] = ["안녕하세요!"]

# 챗봇 대화 컨테이너
response_container = st.container()
# 사용자 입력 컨테이너
container = st.container()

def conversational_chat(query):
    result = chain({"question": query, "chat_history": st.session_state['history']})
    st.session_state['history'].append((query, result["answer"]))
    return result["answer"]

with container:
    with st.form(key='Conv_Question', clear_on_submit=True):
        user_input = st.text_input("Query:", placeholder="강의에 대해 질문해보세요:", key='input')
        submit_button = st.form_submit_button(label='Send')
        
    if submit_button and user_input:
        output = conversational_chat(user_input)
        st.session_state['past'].append(user_input)
        st.session_state['generated'].append(output)

if st.session_state['generated']:
    with response_container:
        for i in range(len(st.session_state['generated'])):
            message(st.session_state["past"][i], is_user=True, key=str(i) + '_user', avatar_style="fun-emoji", seed="Nala")
            message(st.session_state["generated"][i], key=str(i), avatar_style="bottts", seed="Fluffy")


2024-09-20 00:12:40.380 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 00:12:40.380 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 00:12:40.386 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 00:12:40.389 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 00:12:40.391 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 00:12:40.394 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 00:12:40.396 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 00:12:40.397 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar